# Collaborative Filtering (İş Birlikçi Filtreleme)
- Item-Based Collective Filtering
- User-Based Collective Filtering
- Model-Based Collective Filtering

#### 1. Item Based Collective Filtering

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
movie = pd.read_csv('/kaggle/input/recommender/recommender_systems/datasets/movie_lens_dataset/movie.csv')
rating = pd.read_csv('/kaggle/input/recommender/recommender_systems/datasets/movie_lens_dataset/rating.csv')
df = movie.merge(rating, how="left", on="movieId")
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


### 2: User Movie Df'inin Oluşturulması

In [2]:

df.head()
df.shape

(20000797, 6)

In [3]:
df["title"].nunique()

df["title"].value_counts().head()

comment_counts = pd.DataFrame(df["title"].value_counts())
rare_movies = comment_counts[comment_counts["title"] <= 1000].index
common_movies = df[~df["title"].isin(rare_movies)]
common_movies.shape
common_movies["title"].nunique()
df["title"].nunique()

user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")

user_movie_df.shape
user_movie_df.columns

Index([''burbs, The (1989)', '(500) Days of Summer (2009)',
       '*batteries not included (1987)', '...And Justice for All (1979)',
       '10 Things I Hate About You (1999)', '10,000 BC (2008)',
       '101 Dalmatians (1996)',
       '101 Dalmatians (One Hundred and One Dalmatians) (1961)',
       '102 Dalmatians (2000)', '12 Angry Men (1957)',
       ...
       'Zero Dark Thirty (2012)', 'Zero Effect (1998)', 'Zodiac (2007)',
       'Zombieland (2009)', 'Zoolander (2001)', 'Zulu (1964)', '[REC] (2007)',
       'eXistenZ (1999)', 'xXx (2002)', '¡Three Amigos! (1986)'],
      dtype='object', name='title', length=3159)

#### Adım 3: Item-Based Film Önerilerinin Yapılması

In [4]:
movie_name = "Matrix, The (1999)"
movie_name = "Ocean's Twelve (2004)"
movie_name = user_movie_df[movie_name]
user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)


movie_name = pd.Series(user_movie_df.columns).sample(1).values[0]
movie_name = user_movie_df[movie_name]
user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)


def check_film(keyword, user_movie_df):
    return [col for col in user_movie_df.columns if keyword in col]

check_film("Insomnia", user_movie_df)

['Insomnia (1997)', 'Insomnia (2002)']

#### Adım 4: Çalışma Scriptinin Hazırlanması

In [5]:
def create_user_movie_df():
    import pandas as pd
    movie = pd.read_csv('/kaggle/input/recommender/recommender_systems/datasets/movie_lens_dataset/movie.csv')
    rating = pd.read_csv('/kaggle/input/recommender/recommender_systems/datasets/movie_lens_dataset/rating.csv')
    df = movie.merge(rating, how="left", on="movieId")
    comment_counts = pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 1000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df

In [6]:
user_movie_df = create_user_movie_df()

In [7]:
def item_based_recommender(movie_name, user_movie_df):
    movie_name = user_movie_df[movie_name]
    return user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

In [8]:
item_based_recommender("Matrix, The (1999)", user_movie_df)

movie_name = pd.Series(user_movie_df.columns).sample(1).values[0]

item_based_recommender(movie_name, user_movie_df)

title
Motorcycle Diaries, The (Diarios de motocicleta) (2004)                            1.000000
French Twist (Gazon maudit) (1995)                                                 0.606863
Associate, The (1996)                                                              0.563669
Auntie Mame (1958)                                                                 0.552423
Crossing Guard, The (1995)                                                         0.529065
Armour of God II: Operation Condor (Operation Condor) (Fei ying gai wak) (1991)    0.522591
Kagemusha (1980)                                                                   0.520201
War, The (1994)                                                                    0.513198
Before and After (1996)                                                            0.511177
Tender Mercies (1983)                                                              0.507952
dtype: float64